In [1]:
import math
import random
import numpy as np
import pandas as pd

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

#env = gym.make('CartPole-v0').unwrapped

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# if gpu is to be used
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
device

#import os
#pathName = '/home/ubuntu/data/PEC/model_dict/ml100k/'
#if not os.path.exists(pathName):
#     os.makedirs(pathName)
#MODELPATH = pathName + 'dnn_v1.0_'

data_path = '/home/zhangxz/workspace/dataset/R3009_U5_V100/'
UIT = pd.read_csv(data_path + 'UIT.csv')
simUsers = np.loadtxt(data_path+"simUser.csv",  delimiter=',',dtype=int)

trainUIT = UIT[UIT['day']<max(UIT['day']+1)*0.6]
validUIT = UIT[UIT['day']<max(UIT['day']+1)*0.8]

contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())

rewardPara = {"alpha":0.01,"betao":1,"betal":1}
latency = [0.2,1,0.8]
Bu = 20

BATCH_SIZE = 128
GAMMA = 0.99

num_episodes = 10
EPS_START = 0.99
EPS_END = 0.01
EPS_DECAY = trainUIT.shape[0]*3
agentStep = 0

torch.cuda.empty_cache()
UIT.shape,trainUIT.shape,validUIT.shape

((300983, 12), (198175, 12), (253630, 12))

In [2]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        #self.embedding = nn.Embedding(input_dim, emb_dim)
        
        #self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        #embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        
        embedded = self.dropout(src)
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell
    
    
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        #self.embedding = nn.Embedding(output_dim, emb_dim)
        
        #self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.rnn = nn.LSTM(output_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input_ = input_.float().unsqueeze(0)
        #print("decode input shape",input_.shape)
        #input = [1, batch size]
        
        #embedded = self.dropout(self.embedding(input))
        embedded = self.dropout(input_)
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        #prediction = self.fc_out(output.squeeze(0)).unsqueeze(-1)
        prediction = self.fc_out(output.squeeze(0))
        #print(prediction.shape)
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [3]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        #print("trg_len",trg.shape[0],"batch_size",trg.shape[1])
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        #print(input.shape)
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            #top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            #print(trg[t].shape,output.unsqueeze(-1).shape)
            input = trg[t] if teacher_force else output
        
        return outputs

In [4]:
INPUT_DIM = 1
OUTPUT_DIM = 1
ENC_EMB_DIM = 0
DEC_EMB_DIM = 0
HID_DIM = 64
#HID_DIM = 64

N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [5]:
FOLD = 2
CITY = 0
BATCHSIZE = contentNum

model.load_state_dict(torch.load("fold{}-city{}-model.pt".format(FOLD,CITY)))      

<All keys matched successfully>

In [6]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum,dtype=np.float32)
        self.e = np.zeros(shape=contentNum,dtype=int)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency,dtype=np.float32)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
        self.r[u] =  self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (self.r,
                self.p, 
                self.e,
                self.S,
                self.l)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()

class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
        #self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit,dpc,data):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1
        
            if data != None:
                actionIndex = self.evaluate(dpc,data,self.Bu+1)
                if self.W[-1] not in actionIndex:
                    actionIndex.pop() 
                for index in actionIndex:
                    action[index] = 1
                env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action
    def evaluate(self,model, batch, cachesize):

        model.eval()

        with torch.no_grad():

            src = batch.permute(1,0,2).float().to(device)

            batchsize = src.shape[1]
            trg = torch.zeros((26,batchsize,1)).to(device)

            torch.cuda.empty_cache()

            output = model(src, trg, 0) #turn off teacher forcing

            output = output.squeeze(-1)

            output = output[0].cpu().numpy()

            max_item=np.argsort(-output)[0:cachesize]
            #print(max_item,list(max_item))
        return list(max_item)

In [7]:
UIT

,u,i,day,time,viewtime,video_type,video_format,city,city_isp,client_ip,conn_type,device_type
0,365,3391,0,0,148,11,21,0,0,8812,1,0
1,203,5779,0,0,7,11,4,0,0,9063,1,0
2,208,4675,0,0,92,13,4,0,0,3444,1,0
3,159,332,0,0,56,11,3,0,0,4058,1,0
4,50,674,0,0,439,11,4,0,0,1549,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
300978,483,6831,29,2591880,34,11,4,0,0,7061,1,0
300979,158,8448,29,2591880,34,11,4,0,0,11316,1,0
300980,483,6463,29,2591940,35,11,4,0,0,7061,1,0
300981,158,4715,29,2591940,34,11,4,0,0,11316,1,0


In [8]:
UIT['day'] = UIT['time']//(60*60)
UIT

,u,i,day,time,viewtime,video_type,video_format,city,city_isp,client_ip,conn_type,device_type
0,365,3391,0,0,148,11,21,0,0,8812,1,0
1,203,5779,0,0,7,11,4,0,0,9063,1,0
2,208,4675,0,0,92,13,4,0,0,3444,1,0
3,159,332,0,0,56,11,3,0,0,4058,1,0
4,50,674,0,0,439,11,4,0,0,1549,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
300978,483,6831,719,2591880,34,11,4,0,0,7061,1,0
300979,158,8448,719,2591880,34,11,4,0,0,11316,1,0
300980,483,6463,719,2591940,35,11,4,0,0,7061,1,0
300981,158,4715,719,2591940,34,11,4,0,0,11316,1,0


In [9]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

Frq_hour = torch.zeros((BATCHSIZE,720*6)).to(device)
m = 20


rewardPara = {"alpha":1,"betao":1,"betal":1}

sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)
i_episode = 0

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    if uit[1] in np.argwhere(ue.lastAction):
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    if uit[2]>=m:
        test_iterator = Frq_hour[:,uit[2]-m:uit[2]]/Frq_hour[:,uit[2]-m:uit[2]].sum(dim=0)
        test_iterator = test_iterator.unsqueeze(2)
        #print(test_iterator.sum())
        
    else:
        test_iterator = None
    
    ue.selectAction(env,uit,model,test_iterator)

    for content in np.argwhere(ue.lastAction):
        Frq_hour[content,uit[2]]+=1
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sun Oct 17 00:18:02 2021   Index: 9999   Loss: 0.0 --
Reward: [-0.75377  0.02616  0.     ] total reward: -0.72761
UEHitrate: 0.0  edgeHitrate 0.0327 sumHitrate 0.0327  privacy: 0.43735

--Time: Sun Oct 17 00:31:22 2021   Index: 19999   Loss: 0.0 --
Reward: [-2.43069  0.19496  0.     ] total reward: -2.23573
UEHitrate: 0.0  edgeHitrate 0.2437 sumHitrate 0.2437  privacy: 0.64891

--Time: Sun Oct 17 00:44:41 2021   Index: 29999   Loss: 0.0 --
Reward: [-2.73119  0.22909  0.     ] total reward: -2.5021
UEHitrate: 0.0  edgeHitrate 0.28637 sumHitrate 0.28637  privacy: 0.72124

--Time: Sun Oct 17 00:57:09 2021   Index: 39999   Loss: 0.0 --
Reward: [-2.79039  0.24174  0.     ] total reward: -2.54865
UEHitrate: 0.0  edgeHitrate 0.30218 sumHitrate 0.30218  privacy: 0.76267

--Time: Sun Oct 17 01:08:01 2021   Index: 49999   Loss: 0.0 --
Reward: [-2.72758  0.25296  0.     ] total reward: -2.47462
UEHitrate: 0.0  edgeHitrate 0.3162 sumHitrate 0.3162  privacy: 0.77896

--Time: Sun Oct 17 01:1

In [12]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.0327 , 0.2437 , 0.28637, 0.30218, 0.3162 , 0.3238 , 0.3243 ,
        0.33385, 0.33649, 0.33967, 0.34322, 0.3454 , 0.34367, 0.34939,
        0.3552 , 0.35403, 0.35844, 0.36196, 0.36425, 0.36559, 0.36829,
        0.36725, 0.36651, 0.36591, 0.36685, 0.36552, 0.36416, 0.36467,
        0.36737, 0.3712 , 0.37152, 0.     ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0.0327 , 0.2437 , 0.28637, 0.30218, 0.3162 , 0.3238 , 0.3243 ,
        0.33385, 0.33649, 0.33967, 0.34322, 0.3454 , 0.34367, 0.34939,
        0.3552 , 0.35403, 0.35844, 0.36196, 0.36425, 0.36559, 0.36829,
        0.36725, 0.36651, 0.36591, 0.36685, 0.36552, 0.36416, 0.36467,
        0.36737, 0.3712 , 0.37152, 0.     ]))

In [13]:
privacyReduction

array([0.43735, 0.64891, 0.72124, 0.76267, 0.77896, 0.80359, 0.82002,
       0.8286 , 0.83705, 0.83736, 0.83769, 0.84429, 0.84554, 0.85112,
       0.85774, 0.86172, 0.86432, 0.85868, 0.85816, 0.85602, 0.83835,
       0.84243, 0.84704, 0.85096, 0.85426, 0.85716, 0.86023, 0.86308,
       0.86552, 0.8674 , 0.86756, 0.     ])

In [14]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum,dtype=np.float32)
        self.e = np.zeros(shape=contentNum,dtype=int)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency,dtype=np.float32)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
        self.r[u] =  self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (self.r,
                self.p, 
                self.e,
                self.S,
                self.l)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()

class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
        #self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit,dpc,data):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1
        
            if data != None:
                actionIndex = self.evaluate(dpc,data,self.Bu+1)
            else:
                actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
            if self.W[-1] not in actionIndex:
                actionIndex.pop() 
            for index in actionIndex:
                action[index] = 1

            env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action
    def evaluate(self,model, batch, cachesize):

        model.eval()

        with torch.no_grad():

            src = batch.permute(1,0,2).float().to(device)

            batchsize = src.shape[1]
            trg = torch.zeros((26,batchsize,1)).to(device)

            torch.cuda.empty_cache()

            output = model(src, trg, 0) #turn off teacher forcing

            output = output.squeeze(-1)

            output = output[0].cpu().numpy()

            max_item=np.argsort(-output)[0:cachesize]
            #print(max_item,list(max_item))
        return list(max_item)

In [15]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

Frq_hour = torch.zeros((BATCHSIZE,720*6)).to(device)
m = 20


rewardPara = {"alpha":1,"betao":1,"betal":1}

sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)
i_episode = 0

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    if uit[1] in np.argwhere(ue.lastAction):
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    if uit[2]>=m:
        test_iterator = Frq_hour[:,uit[2]-m:uit[2]]/Frq_hour[:,uit[2]-m:uit[2]].sum(dim=0)
        test_iterator = test_iterator.unsqueeze(2)
        #print(test_iterator.sum())
        
    else:
        test_iterator = None
    
    ue.selectAction(env,uit,model,test_iterator)

    for content in np.argwhere(ue.lastAction):
        Frq_hour[content,uit[2]]+=1
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sun Oct 17 10:07:39 2021   Index: 9999   Loss: 0.0 --
Reward: [-62.61858   0.08216   0.     ] total reward: -62.53642
UEHitrate: 0.0  edgeHitrate 0.1027 sumHitrate 0.1027  privacy: 3.94465

--Time: Sun Oct 17 10:20:09 2021   Index: 19999   Loss: 0.0 --
Reward: [-32.62463   0.22296   0.     ] total reward: -32.40167
UEHitrate: 0.0  edgeHitrate 0.2787 sumHitrate 0.2787  privacy: 3.15531

--Time: Sun Oct 17 10:31:59 2021   Index: 29999   Loss: 0.0 --
Reward: [-22.26648   0.24776   0.     ] total reward: -22.01872
UEHitrate: 0.0  edgeHitrate 0.3097 sumHitrate 0.3097  privacy: 2.77583

--Time: Sun Oct 17 10:44:09 2021   Index: 39999   Loss: 0.0 --
Reward: [-17.13323   0.25574   0.     ] total reward: -16.87749
UEHitrate: 0.0  edgeHitrate 0.31967 sumHitrate 0.31968  privacy: 2.5636

--Time: Sun Oct 17 10:56:41 2021   Index: 49999   Loss: 0.0 --
Reward: [-14.01512   0.26416   0.     ] total reward: -13.75096
UEHitrate: 0.0  edgeHitrate 0.3302 sumHitrate 0.3302  privacy: 2.4029

--Time

In [16]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.1027 , 0.2787 , 0.3097 , 0.31968, 0.3302 , 0.33547, 0.3343 ,
        0.3426 , 0.34427, 0.34667, 0.34958, 0.35123, 0.34905, 0.35439,
        0.35987, 0.35841, 0.36256, 0.36584, 0.36794, 0.36909, 0.37162,
        0.37043, 0.36956, 0.36882, 0.36965, 0.36822, 0.36675, 0.36717,
        0.36978, 0.37353, 0.37384, 0.     ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0.1027 , 0.2787 , 0.3097 , 0.31967, 0.3302 , 0.33547, 0.3343 ,
        0.3426 , 0.34427, 0.34667, 0.34958, 0.35123, 0.34905, 0.35439,
        0.35987, 0.35841, 0.36256, 0.36584, 0.36794, 0.36909, 0.37162,
        0.37043, 0.36956, 0.36883, 0.36965, 0.36822, 0.36675, 0.36717,
        0.36978, 0.37353, 0.37384, 0.     ]))

In [17]:
privacyReduction

array([3.94465, 3.15531, 2.77583, 2.5636 , 2.4029 , 2.29392, 2.2119 ,
       2.14106, 2.07378, 1.99797, 1.94271, 1.90889, 1.86838, 1.84176,
       1.82212, 1.7953 , 1.77474, 1.74094, 1.71988, 1.69583, 1.64948,
       1.63865, 1.62586, 1.6169 , 1.60787, 1.59832, 1.58801, 1.57983,
       1.57357, 1.56894, 1.56854, 0.     ])